### CICModbus2023 Flow labeling 

In [143]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import glob
import os
from sys import platform
import datetime

datasets = []

root_dir = "./ModbusDataset"

def find_csv_files_in_folder(start_path,folder_name):
    csv_files = []
    for root, dirs, files in os.walk(start_path):
        # Check if "attack logs" or "attacker logs" is in the path
        if folder_name in root.split(os.sep) :
            csv_files.extend([os.path.join(root, f) for f in files if f.endswith('.csv')])
    return csv_files

def print_csv(datasets_dir_list,replace_dir):
    for i,dataset in enumerate(datasets_dir_list) :
        print(i+1,dataset.replace(replace_dir,""))

datasets_dir = find_csv_files_in_folder(root_dir,"output")
attack_logs_dir = find_csv_files_in_folder(root_dir,"attack logs")
attack_logs_dir.extend(find_csv_files_in_folder(root_dir,"attacker logs"))
benign_datasets_dir = [ds for ds in datasets_dir  if ds.find("benign")!=(-1)]
attack_dataset_dir = [ds for ds in datasets_dir  if ds.find("attack")!=(-1)]
print("num of total fix-ordered dataset files ",len(datasets_dir))
print("num of benign dataset files",len(benign_datasets_dir))
print("num of attack dataset files",len(attack_dataset_dir))
print("num of attack logs ",len(attack_logs_dir))


num of total fix-ordered dataset files  170
num of benign dataset files 62
num of attack dataset files 108
num of attack logs  34


In [145]:

print("list of csv dataset directories under ./ModbusDataset/*")
print_csv(datasets_dir,"./ModbusDataset/")

list of csv dataset directories under ./ModbusDataset/*
1 attack/external/network-wide/output/network-wide-normal-1_fix_ord.pcap_Flow.csv
2 attack/external/network-wide/output/network-wide-normal-0_fix_ord.pcap_Flow.csv
3 attack/external/ied4c/ied4c-network-capture/output/veth8bc3408-0_fix_ord.pcap_Flow.csv
4 attack/external/external-attacker/external-attacker-network-capture/output/veth665f3cf-0_fix_ord.pcap_Flow.csv
5 attack/external/ied1b/ied1b-network-capture/output/vethd9e14c0-0_fix_ord.pcap_Flow.csv
6 attack/external/scada-hmi/scada-hmi-network-capture/output/veth5bbeaa2-0_fix_ord.pcap_Flow.csv
7 attack/external/central-agent/central-agent-network-capture/output/veth460b141-0_fix_ord.pcap_Flow.csv
8 attack/external/ied1a/ied1a-network-capture/output/veth4edc015-0_fix_ord.pcap_Flow.csv
9 attack/compromised-scada/ied4c/ied4c-network-captures/output/vethe685ac9-4_fix_ord.pcap_Flow.csv
10 attack/compromised-scada/ied4c/ied4c-network-captures/output/vethe685ac9-3_fix_ord.pcap_Flow.csv

In [146]:

print("list of csv attack logs directories under (.ModbusDataset/attack/*)")
print_csv(attack_logs_dir,"./ModbusDataset/attack/")

list of csv attack logs directories under (.ModbusDataset/attack/*)
1 compromised-scada/attack logs/03-18-2023/03-18-2023-1.csv
2 compromised-scada/attack logs/03-16-2023/03-16-2023-1.csv
3 compromised-scada/attack logs/03-16-2023/03-16-2023-2.csv
4 compromised-scada/attack logs/03-16-2023/03-16-2023-3.csv
5 compromised-scada/attack logs/03-20-2023/03-20-2023-1.csv
6 compromised-scada/attack logs/03-22-2023/03-22-2023-1.csv
7 compromised-scada/attack logs/03-14-2023/03-14-2023-2.csv
8 compromised-scada/attack logs/03-14-2023/03-14-2023-1.csv
9 compromised-scada/attack logs/03-14-2023/03-14-2023-3.csv
10 compromised-scada/attack logs/03-15-2023/03-15-2023-2.csv
11 compromised-scada/attack logs/03-15-2023/03-15-2023-3.csv
12 compromised-scada/attack logs/03-15-2023/03-15-2023-1.csv
13 compromised-scada/attack logs/03-17-2023/03-17-2023-2.csv
14 compromised-scada/attack logs/03-17-2023/03-17-2023-1.csv
15 compromised-scada/attack logs/03-13-2023/03-13-2023-1.csv
16 compromised-scada/attac

#### Labeling functions


In [164]:
# Basic preprocessing before getting started on labelling.
# Deletes rows with "Infinity" and NaNs, converts "Timestamp" to Pandas Datetime, and converts all necessary columns to numeric values
# throw away 'Src IP', 'Dst IP'
# Attempted-Category ,Int_64 Columns label wasn't considered 

print_index = False

def format_csv_for_labelling(df):
    df = df.replace('Infinity', np.nan)
    # Clean the Timestamp strings to always include microseconds (append .0 if missing)
    df['Timestamp'] = pd.to_datetime(
        df['Timestamp'].apply(
            lambda x: x if '.' in x.split()[-1] else f"{x}.0"  # Split into date/time and check time part
        ),
        format='%Y-%m-%d %H:%M:%S.%f'  # Parse with microseconds
    )
    for column in df.columns:
        if column not in ['Flow ID' , 'Timestamp', 'Src IP', 'Dst IP', 'Label']:
            df[column] = pd.to_numeric(df[column], errors='coerce')
    df.dropna()
    return df.dropna()

def read_csvs_from_path_and_reformat(path):
    df = pd.read_csv(path, encoding='cp1252')

    df = format_csv_for_labelling(df)
    print("labels after pre-processing:\n", df["Label"].value_counts())

    int32_columns = ["Src Port", "Dst Port", "Flow Duration", "Total Fwd Packet", "Total Bwd packets", "Total Length of Fwd Packet", "Total Length of Bwd Packet", "Fwd Packet Length Max",
        "Fwd Packet Length Min", "Bwd Packet Length Max", "Bwd Packet Length Min", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total",
        "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "Packet Length Min", "Packet Length Max", "FIN Flag Count", "SYN Flag Count", "RST Flag Count", "PSH Flag Count",
        "ACK Flag Count", "URG Flag Count", "CWR Flag Count", "ECE Flag Count", "Subflow Fwd Packets", "Subflow Fwd Bytes",
        "Subflow Bwd Packets", "Subflow Bwd Bytes", "FWD Init Win Bytes", "Bwd Init Win Bytes", "Fwd Act Data Pkts", "Fwd Seg Size Min", "Active Max",
        "Active Min", "Idle Max", "Idle Min"]

    int16_columns = ["Fwd Header Length", "Bwd Header Length", "ICMP Code", "ICMP Type"]

    for column in int32_columns:
        df[column] = df[column].astype('int32')

    for column in int16_columns:
        df[column] = df[column].astype('int16')

    return df

# Main labelling function. Only used for labelling Malicious flows.
# Timestamps are in NANOSECONDS (!) Unix time. Note that the CSV files are in the UTC timezone.
# df = dataframe with flows. Note that labelling happens inplace on the 'df' parameter, and so this function doesn't return anything
# label = the label that will be given to flows matching the criteria specified in the function
# additional_filters = add any additional constraints that cannot be covered by the already provided function arguments
# There is no attempted_category label 
# for details on how the "Attempted" categories are defined:  (https://intrusion-detection.distrinet-research.be/CNS2022/Tools_Documentation.html)
# payload_filter = When set to true, this will automatically add a constraint ["Total Length of Fwd Packet"] == 0. Note that

def label_flows(df, label, attack_start_time_nanoseconds, attack_end_time_nanoseconds, src_ip_list=None,
                dst_ip_list= None, src_port_list=None, dst_port_list=None, additional_filters=[], attempted_category=-1, payload_filter=False):

    # Create initial mask for whole df with all values set to True. Squeeze is necessary to remove second axis (with value 1)
    # The reason is that a df of shape (X,) gets converted to (1,X) if you '&' it with a df of shape (X,1)
    mask = pd.DataFrame(True,index=df.index,columns=[df.columns[0]]).squeeze()

    attack_start_datetime = pd.to_datetime(attack_start_time_nanoseconds, unit='ns')
    attack_end_datetime = pd.to_datetime(attack_end_time_nanoseconds, unit='ns')

    mask &= (df["Timestamp"] >= attack_start_datetime)
    mask &= (df["Timestamp"] <= attack_end_datetime)

    if src_ip_list is not None:
        mask &= (df["Src IP"].isin(src_ip_list))
    if dst_ip_list is not None:
        mask &= (df["Dst IP"].isin(dst_ip_list))

    if src_port_list is not None:
        mask &= (df["Src Port"].isin(src_port_list))
    if dst_port_list is not None:
        mask &= (df["Dst Port"].isin(dst_port_list))

    if payload_filter:
        mask &= (df["Total Length of Fwd Packet"] == 0)

    for filter in additional_filters:
        mask &= filter

    df["Label"].mask(mask, label, inplace=True)
    df["Attempted Category"].mask(mask, attempted_category, inplace=True)

# This function is called when all labelling of malicious flows is completed. Anything that has not yet received a label
# so far is labelled as Benign.
def label_rest_as_benign_and_write_csv(df, input_path):
    
    df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN", inplace=True)
    print("label count after labelling:\r\n", df["Label"].value_counts())
    # Construct the ready directory path alongside output
    ready_dir = os.path.join(os.path.dirname(os.path.dirname(input_path)), 'ready')
    os.makedirs(ready_dir, exist_ok=True)
    filename = os.path.basename(dir)
    name, ext = os.path.splitext(filename)  # Split the extension (name, ".csv")
    cleaned_name = name.replace("_fix_ord.pcap_Flow", "-labeled")
    # Construct full output path
    output_path = os.path.join(ready_dir, f"{cleaned_name}{ext}")  # Append extension
    # Adds line numbers in the first column if print_index is set to true
    if print_index:
        df.reset_index(inplace=True, drop=True)
        df.index += 1
        df.index.name = 'id'
        df.to_csv(output_path)
    else:
        df.to_csv(output_path, index=False)


In [165]:
for dir in benign_datasets_dir:
    print(dir)
    label_rest_as_benign_and_write_csv(read_csvs_from_path_and_reformat(dir) ,dir)
            

./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-18_fix_ord.pcap_Flow.csv
labels after pre-processing:
 Label
NeedManualLabel    153041
Name: count, dtype: int64
label count after labelling:
 Label
BENIGN    153041
Name: count, dtype: int64


/tmp/ipykernel_34873/2826604722.py:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN", inplace=True)


./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-22_fix_ord.pcap_Flow.csv
labels after pre-processing:
 Label
NeedManualLabel    153868
Name: count, dtype: int64
label count after labelling:
 Label
BENIGN    153868
Name: count, dtype: int64


/tmp/ipykernel_34873/2826604722.py:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN", inplace=True)


./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-30_fix_ord.pcap_Flow.csv
labels after pre-processing:
 Label
NeedManualLabel    152394
Name: count, dtype: int64
label count after labelling:
 Label
BENIGN    152394
Name: count, dtype: int64


...

/tmp/ipykernel_34873/2826604722.py:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN", inplace=True)


./ModbusDataset/benign/ied1a/ied1a-network-capture/output/veth4edc015-normal-4_fix_ord.pcap_Flow.csv
labels after pre-processing:
 Label
NeedManualLabel    167254
Name: count, dtype: int64
label count after labelling:
 Label
BENIGN    167254
Name: count, dtype: int64


/tmp/ipykernel_34873/2826604722.py:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN", inplace=True)


./ModbusDataset/benign/ied1a/ied1a-network-capture/output/veth4edc015-normal-8_fix_ord.pcap_Flow.csv
labels after pre-processing:
 Label
NeedManualLabel    169112
Name: count, dtype: int64
label count after labelling:
 Label
BENIGN    169112
Name: count, dtype: int64


/tmp/ipykernel_34873/2826604722.py:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN", inplace=True)


#### External Attacks :
- Attacks from an unknown IP address (185.175.0.7) targeting 185.175.0.4.
#### Compromised SCADA Attacks :
- Attacks from 185.175.0.3 targeting 185.175.0.4 and 185.175.0.8.
- Days 12 to 15 missed in attack logs.
#### Compromised IED Attacks
- Attacks from 185.175.0.5 targeting 185.175.0.2.
